In [1]:
!pip install pandas
!pip install openpyxl
!pip install yfiles_jupyter_graphs
!pip install networkx



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import os
from yfiles_jupyter_graphs import GraphWidget


In [3]:
notebook_path = os.path.abspath("notebook.ipynb")
file_path = os.path.join(os.path.dirname(notebook_path), "source_data/berber_1.xlsx")

df = pd.read_excel(file_path)

In [4]:

# shows empty widget
w = GraphWidget()

w.orthogonal_layout()

In [26]:
#for (index, colname) in enumerate(df):
#  if index > 2:
#    print(index, df[colname].values[2])

tp_index = 1
timepoints = []
visits = []
activities = []
activity_bc = {}
activity_row = []

activity = None
for index, col_def in df.iterrows():
  if index > 5:
    activity_present = not pd.isnull(df.iloc[index, 1])
    if activity_present:
      activity = df.iloc[index, 1]
      activities.append(activity)
    activity_row.append({ 'name': activity, 'present': activity_present })
    bc_present = not pd.isnull(df.iloc[index, 2])
    if bc_present:
      if not activity in activity_bc:
        activity_bc[activity] = { 'bc': [] }  
      activity_bc[activity]['bc'].append(df.iloc[index, 2])
print("A1:", activities)
print("A2:", activity_bc)
print("A3:", activity_row)

for index in range(df.shape[1]):
  if index > 2:
    window = df.iloc[4, index]
    window_is_null = pd.isnull(df.iloc[4,index])
    if window_is_null:
      window = ""
    visits.append({ 'window': window })
    timepoints.append({ 'name': "tp_%s" % (tp_index)})
    tp_index += 1
print("T:", timepoints)
print("V:", visits)

tp_activity = {}
for tp in timepoints:
  tp_name = tp['name']
  tp_activity[tp_name] = {}
  for activity in activities:
    tp_activity[tp_name][activity] = False
for index in range(df.shape[1]):
  if index > 2:
    column = df.iloc[:, index]
    row = 0
    for x in column:
      if row >= 6:
        if not pd.isnull(x):
          #print("X ROW:", index, row, x)
          if x.upper() == "X":
            if activity_row[row - 6]['present']:
              activity = activity_row[row - 6]['name']
              tp = timepoints[index - 3]['name']
              tp_activity[tp][activity] = True
      row += 1
print("TPA:", tp_activity) 

A1: ['Informed Consent', 'Inclusion / Exclusion Criteria', 'Physical Examination']
A2: {'Physical Examination': {'bc': ['PE 1', 'PE 2', 'Weight', 'Height']}}
A3: [{'name': 'Informed Consent', 'present': True}, {'name': 'Inclusion / Exclusion Criteria', 'present': True}, {'name': 'Physical Examination', 'present': True}, {'name': 'Physical Examination', 'present': False}, {'name': 'Physical Examination', 'present': False}, {'name': 'Physical Examination', 'present': False}]
T: [{'name': 'tp_1'}, {'name': 'tp_2'}, {'name': 'tp_3'}, {'name': 'tp_4'}, {'name': 'tp_5'}, {'name': 'tp_6'}, {'name': 'tp_7'}, {'name': 'tp_8'}, {'name': 'tp_9'}, {'name': 'tp_10'}, {'name': 'tp_11'}]
V: [{'window': ''}, {'window': ''}, {'window': '-2..2 Days'}, {'window': '-2..2 Days'}, {'window': '-2..2 Days'}, {'window': '-2..2 Days'}, {'window': '-2..2 Days'}, {'window': '-2..2 Days'}, {'window': '-2..2 Days'}, {'window': '0..5 Days'}, {'window': '0..2 Weeks'}]
TPA: {'tp_1': {'Informed Consent': True, 'Inclusi

In [ ]:
nodes = [
  {'id': 0, 'properties': {'label': 'Entry'}},
  {'id': 99999, 'properties': {'label': 'Exit'}}
]
edges = []
node_index = 1
edge_index = 0
prev_node = None
for tp in timepoints:
  nodes.append({ 'id': node_index, 'properties': {'label': tp['name']} })
  if prev_node == None:
    edges.append( {id: edge_index, 'start': 0, 'end': node_index, 'properties': {'label': "entry"}})
  else:
    edges.append( {id: edge_index, 'start': prev_node, 'end': node_index, 'properties': {'label': "next"}})
  prev_node = node_index
  node_index += 1
  edge_index += 1
edges.append( {id: edge_index, 'start': prev_node, 'end': 99999, 'properties': {'label': "exit"}})
w.set_nodes(nodes)
w.set_edges(edges)
w

In [6]:
a = w.get_graph_layout()
print(a)

{'algorithm': 'orthogonal', 'options': {}}
